In [5]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

загрузим датасет MNIST и разделим данные

In [3]:

from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype("int")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

нормируем данные

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

обучение MLP и вывод метрик

In [17]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
model.fit(X_train, y_train)

preds = model.predict(X_test)
print('MLP Accuracy:', accuracy_score(y_test, preds))
print('MLP F1 micro:', f1_score(y_test, preds, average='micro'))
print('MLP F1 macro:', f1_score(y_test, preds, average='macro'))

MLP Accuracy: 0.9723571428571428
MLP F1 micro: 0.9723571428571428
MLP F1 macro: 0.9721766965677402


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

загрузка датасета и разделение данных

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_data = datasets.MNIST(root="./data", train=False, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)

In [ ]:
CNN

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)

        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
обучение модели

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.0095
Epoch [2/5], Loss: 0.0118
Epoch [3/5], Loss: 0.0342
Epoch [4/5], Loss: 0.0164
Epoch [5/5], Loss: 0.0015


In [15]:
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)              
        _, preds = torch.max(outputs, 1)     

        all_preds.append(preds.cpu().numpy())
        all_targets.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_targets = np.concatenate(all_targets)

вывод метрик

In [16]:
print('CNN Accuracy:', accuracy_score(all_targets, all_preds))
print('CNN F1 micro:', f1_score(all_targets, all_preds, average='micro'))
print('CNN F1 macro:', f1_score(all_targets, all_preds, average='macro'))

CNN Accuracy: 0.9849
CNN F1 micro: 0.9849
CNN F1 macro: 0.9846739766289708


ВЫВОД:
CNN демонстрирует несколько лучшие значения метрик Accuracy и F1-score по сравнению MLP.
Это связано с тем, что CNN учитывает пространственную структуру изображений и эффективно извлекает локальные признаки, в то время как MLP работает с изображением как с плоским вектором признаков.